In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import pickle

alphabet = 'abcdefghijklmnopqrstuvwxyz'.upper()
games = ['WC1','WC2','WC2X','WC3','WC3X','WoW','WoWBC','WoWWotLK','WoWC','WoWMoP','WoWWoD','WoWL','WoWBfA','WoWS','WoWD']

In [2]:
def get_metadata():
    meta_data = []
    
    # get meta data of all major characters

    url = "https://wowpedia.fandom.com/wiki/Major_characters"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    soup_a = soup.select("p > small")
    soup_b = soup.select("p > a")
    for i,s in enumerate(soup_a):
        d = {}

        expansions = s.text.replace(",","").replace("—","").split()
        expansions = [int(1) if i in expansions else int(0) for i in games]

        text = soup_b[i]['href'][6:]
        name = text.replace("%27", "'")

        if 1 in expansions:
            d['Name'] = name
            for j,g in enumerate(games):
                d[g] = int(expansions[j])

    
            meta_data.append(d)
    
    return meta_data

In [3]:
meta_data = get_metadata()
df_meta = pd.DataFrame(meta_data, dtype=object)
# saving the df's
with open('df_meta.pickle', 'wb') as handle:
    pickle.dump(df_meta, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_meta

,Name,WC1,WC2,WC2X,WC3,WC3X,WoW,WoWBC,WoWWotLK,WoWC,WoWMoP,WoWWoD,WoWL,WoWBfA,WoWS,WoWD
0,Terokk,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,Talon_King_Ikiss,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,Archimonde,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0
3,Kil'jaeden,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0
4,Mannoroth,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,Zul,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
268,Talanji,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
269,Vilnak'dor,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
270,Helya,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
def get_characters():
    characters = []
    
    # get meta data of all major characters

    url = "https://wowpedia.fandom.com/wiki/Major_characters"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    soup_a = soup.select("p > small")
    soup_b = soup.select("p > a")
    for i,s in enumerate(soup_a):
        d = {}

        expansions = s.text.replace(",","").replace("—","").split()
        expansions = [int(1) if i in expansions else int(0) for i in games]

        text = soup_b[i]['href'][6:]
        name = text.replace("%27", "'")

        if 1 in expansions:
            d['Name'] = name
            
            characters.append(d)
    # get lore characters

    c_list = []

    for letter in tqdm(alphabet):
        url = "https://wowpedia.fandom.com/wiki/Category:Lore_characters?pagefrom={}".format(letter)
        html = urlopen(url).read()
        soup = BeautifulSoup(html, features="html.parser")

        for i in soup.find_all('li'):
            text = i.find('a')['href']
            verboten = ['/wiki/Genealogy', '/wiki/Local_Sitemap']
            if text[0:5] == '/wiki' and ":" not in text and text not in verboten:
                c_list.append(text.replace("%27", "'").replace("%22", '"').split("/wiki/")[1])

    c_list = sorted(list(set(c_list)))

    for name in c_list:
        d = {}
        d['Name'] = name

        characters.append(d)
    
    return characters

In [7]:
characters = get_characters()
df = pd.DataFrame(characters)
# saving the df's
with open('df.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)
df


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:14<00:00,  1.73it/s]


,Name
0,Terokk
1,Talon_King_Ikiss
2,Archimonde
3,Kil'jaeden
4,Mannoroth
...,...
1967,Zur'ak_Firefist
1968,Zuura
1969,Zuuzuu
1970,Zuzak
